In [573]:
# Import libraries

# General
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, PolynomialFeatures, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, make_pipeline # Same, but with the latter it is not necessary to name estimator and transformer
#from imblearn.pipeline import Pipeline as Imb_Pipe
from sklearn.compose import ColumnTransformer
from sklearn.cluster import DBSCAN

# Feature Selection
from sklearn.feature_selection import SelectKBest, chi2, f_classif, GenericUnivariateSelect, mutual_info_classif
import eli5

# Predictive Modeling (Models)
from sklearn.dummy import DummyClassifier, DummyRegressor
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_predict, cross_val_score, cross_validate, KFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, NuSVC
from sklearn.linear_model import LinearRegression, LogisticRegression, PassiveAggressiveRegressor, ElasticNet, SGDRegressor, RANSACRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor, VotingClassifier, RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, IsolationForest
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from xgboost import XGBClassifier, XGBRegressor
from scipy.stats import randint

# Evaluation Metrics
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer, fbeta_score, accuracy_score, confusion_matrix, f1_score, precision_recall_curve, recall_score, precision_score
from scipy.sparse import csr_matrix

In [574]:
# Import data_engineered
data = pd.read_pickle("saves/data_engineered.pkl")

In [575]:
# Alternative: Import from csv
#data_types_engineered = pd.read_csv('saves/types_engineered.csv')['types']
#data = pd.read_csv("saves/data_engineered.csv", dtype=data_types_engineered.to_dict())
#data.set_index('id', inplace=True)

In [582]:
# Dashboard
target = 'occupancy_class' # for regression: 'occupancy_rate', 'price_log' | for classification: 'occupancy_class'
drop_cols = ['occupancy_rate'] # additional columns to drop
scoring = 'f1' # for regression: 'neg_mean_squared_error' | for classification: "f1", "recall", "precision", "accuracy", "roc_auc"
test_size = 0.2
random_state = 42

# Preprocessing (Train/Test Split and Pipeline)

## Preprocessing pipeline

In [578]:
# Create list for categorical predictors/features (used in "Scaling with Preprocessing Pipeline") 
cat_features = list(data.columns[data.dtypes==object])
#cat_features.remove("neighbourhood")
#cat_features.remove("zipcode")
cat_features

['cancellation_policy', 'property_type', 'room_type']

In [579]:
# Create list for numerical predictors/features (removing target column, used in "Scaling with Preprocessing Pipeline")
num_features = list(data.columns[data.dtypes!=object])
num_features.remove(target)
num_features

['accommodates_per_bed',
 'accommodates_per_room',
 'am_balcony',
 'am_breakfast',
 'am_child_friendly',
 'am_elevator',
 'am_essentials',
 'am_nature_and_views',
 'am_pets_allowed',
 'am_private_entrance',
 'am_smoking_allowed',
 'am_tv',
 'am_white_goods',
 'bathrooms_log',
 'bedrooms',
 'binary_chg',
 'calculated_host_listings_count',
 'host_is_superhost',
 'instant_bookable',
 'latitude',
 'longitude',
 'maximum_nights',
 'minimum_nights_chg',
 'minimum_nights_log',
 'numeric_chg',
 'price_chg_2020_01',
 'price_extra_fees_sqrt',
 'price_extra_people',
 'price_log',
 'review_scores_rating_sqrt',
 'text_len_chg',
 'text_len_sqrt',
 'wk_mth_discount']

In [580]:
# Build preprocessor pipeline
# Pipeline for numerical features
num_pipeline = Pipeline([
    ('imputer_num', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

# Pipeline for categorical features 
cat_pipeline = Pipeline([
    ('imputer_cat', SimpleImputer(strategy='constant', fill_value='missing')),
    ('1hot', OneHotEncoder(drop='first', handle_unknown='error'))
])

# Complete pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

In [581]:
# Function for getting column names after preprocessing
def get_column_names_from_ColumnTransformer(column_transformer):    
    col_name = []
    for transformer_in_columns in column_transformer.transformers_[:-1]:#the last transformer is ColumnTransformer's 'remainder'
        raw_col_name = transformer_in_columns[2]
        if isinstance(transformer_in_columns[1],Pipeline): 
            transformer = transformer_in_columns[1].steps[-1][1]
        else:
            transformer = transformer_in_columns[1]
        try:
            names = transformer.get_feature_names()
        except AttributeError: # if no 'get_feature_names' function, use raw column name
            names = raw_col_name
        if isinstance(names,np.ndarray): # eg.
            col_name += names.tolist()
        elif isinstance(names,list):
            col_name += names    
        elif isinstance(names,str):
            col_name.append(names)
    return col_name

## Train/test split

In [584]:
# Define predictors and target variable
X = data.drop([target], axis=1)
y = data[target]
X = X.drop(drop_cols, axis=1)

In [585]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size,
                                                        random_state=random_state,
                                                        shuffle=True)
#                                                   stratify=y) # Use stratify=y if labels are inbalanced (e.g. most wines are 5 or 6; check with value_counts()!)

In [586]:
# Saving preprocessed X_train and X_test
X_train_prep_preprocessor = preprocessor.fit(X_train)
X_train_prep_cols = get_column_names_from_ColumnTransformer(X_train_prep_preprocessor)

X_train_prep = X_train_prep_preprocessor.transform(X_train)
X_train_num_prep = num_pipeline.fit_transform(X_train[num_features])
X_test_prep = X_train_prep_preprocessor.transform(X_test)

## Outlier Detection

In [490]:
# Preprocess data
#train_outl = num_pipeline.fit_transform(X_train[num_features], y_train)

In [491]:
# Fit DBSCAN model
#outl_model = DBSCAN(eps=3.0, min_samples=10).fit(train_outl)
#outl_labels = outl_model.labels_

In [492]:
# Display results (# of outliers)
#pd.Series(outl_labels).value_counts()

In [493]:
# Illustrate results
#plt.figure(figsize=(10,10))
#
#unique_labels = set(outl_labels)
#colors = ['blue', 'red']
#
#for color,label in zip(colors, unique_labels):
#    sample_mask = [True if l == label else False for l in outl_labels]
#    plt.plot(train_outl[:,0][sample_mask], train_outl[:, 1][sample_mask], 'o', color=color);
#plt.xlabel('accommodates_per_bed');
#plt.ylabel('accommodates_per_room');

Interpreting the results:

- https://www.kaggle.com/kevinarvai/outlier-detection-practice-uni-multivariate
- https://datascience.stackexchange.com/questions/46092/how-do-we-interpret-the-outputs-of-dbscan-clustering

## Perform Feature Selection (add most useful to modeling pipeline)

In [494]:
# Set X_fs to desired variable
X_fs = X_train[num_features]    # X_train_prep, X_train_num_prep, X_train[num_features]
#X_fs = pd.DataFrame(X_fs, columns = X_train_prep_cols)

**GenericUnivariateSelect** (Classification and Regression)

In [495]:
# Apply GenericUnivariateSelect
trans_GUS = GenericUnivariateSelect(score_func=lambda X, y: X.mean(axis=0), mode='k_best', param=15) #mode='percentile', 'k_best'
X_train_GUS = trans_GUS.fit_transform(X_fs, y_train)

**mutual_info_classif** (Classification)

In [498]:
# Fit mutual_info_classif
X_train_mic = mutual_info_classif(X_fs, y_train)

ValueError: Unknown label type: 'continuous'

In [ ]:
# Plot feature importance
plt.subplots(1, figsize=(26, 1))
sns.heatmap(X_train_mic[:, np.newaxis].T, cmap='Blues', cbar=False, linewidths=1, annot=True)
plt.yticks([], [])
plt.gca().set_xticklabels(X_fs.columns, rotation=45, ha='right', fontsize=12)
plt.suptitle("Feature Importance (mutual_info_classif)", fontsize=18, y=1.2)
plt.gcf().subplots_adjust(wspace=0.2)
pass

In [ ]:
# Apply GenericUnivariateSelect to reduce features (optional)
trans_mic = GenericUnivariateSelect(score_func=mutual_info_classif, mode='k_best', param=15) #mode='percentile', 'k_best', 
X_train_mic_GUS = trans_mic.fit_transform(X_fs, y_train)

In [ ]:
# Print kept features
print("We started with {0} features but retained only {1} of them!".format(
    X_fs.shape[1] - 1, X_train_mic_GUS.shape[1]))

columns_retained_Select = X_fs.columns[trans_mic.get_support()].values
pd.DataFrame(X_train_mic_GUS, columns=columns_retained_Select).head()

**chi2** (Classification)

**mutual_info_regression** (Regression)

# Predictive Modeling: Classification ("occupancy_class")

## Apply Classification Models

In [439]:
# Print current setting for TARGET
target_upper = target.upper()
print("You are currently using " + f"\033[1m{target_upper}\033[0m" + " as the target")

You are currently using OCCUPANCY_CLASS as the target


In [440]:
# Select dataset to use
#X_train = X_train_mic_GUS_red       # X_train, X_train_GUS, X_train_mic, ...

In [441]:
# Select models for comparison
models={'Baseline': DummyClassifier(strategy='most_frequent'),
        'LogReg': LogisticRegression(max_iter=1000),
        'KNN': KNeighborsClassifier(),
        'SVC': SVC(kernel='rbf', C=1E6),
        'Decision Tree': DecisionTreeClassifier(criterion="gini", max_depth=3,random_state=random_state),
        'Random Forest': RandomForestClassifier(random_state=random_state, max_features='sqrt', n_jobs=-1),
        'Gradient Boost': GradientBoostingClassifier(random_state=random_state),
        'XGBoost': XGBClassifier(),
        'AdaBoost': AdaBoostClassifier(random_state=random_state)
       }

In [442]:
# Calculate and display results
results = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'R2'])
i = 0
for m in models.items():
    # Building a full pipeline with our preprocessor and a Classifier
    pipe = Pipeline([('preprocessor', preprocessor), (m[0], m[1])])
    # Making predictions on the training set using cross validation as well as calculating the probabilities
    y_train_pred = cross_val_predict(pipe,
                                     X_train,
                                     y_train.values.ravel(),
                                     cv=5,
                                     verbose=4,
                                     n_jobs=-1)
    # Calculating metrices
    temp = pd.DataFrame(
        {
            'Model': m[0],
            'MSE': mean_squared_error(y_train, y_train_pred),
            'RMSE': mean_squared_error(y_train, y_train_pred, squared=False),
            'MAE': mean_absolute_error(y_train, y_train_pred),
            'R2': r2_score(y_train, y_train_pred),
            'Accuracy': accuracy_score(y_train, y_train_pred),
            'Recall': recall_score(y_train, y_train_pred, average="weighted"),
            'Precision': precision_score(y_train, y_train_pred, average="weighted"),
            'F1 Score': f1_score(y_train, y_train_pred, average="weighted")
        },
        index=[i])
    print(f"Confusion Matrix {m[0]}: \n" + str(confusion_matrix(y_train, y_train_pred)))
    i += 1
    results = pd.concat([results, temp])
results

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.4s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.6s finished
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Confusion Matrix Baseline: 
[[5740    0    0    0    0    0]
 [3857    0    0    0    0    0]
 [2090    0    0    0    0    0]
 [1200    0    0    0    0    0]
 [ 855    0    0    0    0    0]
 [2794    0    0    0    0    0]]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.7s remaining:    2.5s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Confusion Matrix LogReg: 
[[4551  887   22    4    1  275]
 [1832 1628   33    4    4  356]
 [ 797  768   52    9    4  460]
 [ 362  325   32    7    2  472]
 [ 204  208   22    4    3  414]
 [ 565  434   47   10    1 1737]]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    7.6s remaining:   11.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

Confusion Matrix XGBoost: 
[[4652  972   10    3    0  103]
 [ 900 2624   61    6    3  263]
 [ 318 1206   85   13    1  467]
 [ 157  495   59   14    1  474]
 [  88  290   34    6    1  436]
 [ 234  528   68   15    0 1949]]


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.7s remaining:    2.5s


Confusion Matrix AdaBoost: 
[[4232 1309   13    2    0  184]
 [ 663 2700  126    9    0  359]
 [ 260 1153  128   23    0  526]
 [ 103  501   70   10    0  516]
 [  62  299   44   10    0  440]
 [ 165  558   90    9    0 1972]]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s finished
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Model,MSE,RMSE,R2,MAE,Accuracy,Recall,Precision,F1 Score
0,Baseline,6.443336,2.538373,-0.916536,1.755382,0.347121,0.347121,0.120493,0.178890
1,LogReg,3.464623,1.861350,-0.030534,1.132620,0.482463,0.482463,0.413757,0.415630
2,KNN,3.545053,1.882831,-0.054457,1.198657,0.432753,0.432753,0.385876,0.397818
3,SVC,3.398948,1.843624,-0.010999,1.203495,0.409591,0.409591,0.399741,0.403270
4,Decision Tree,2.850871,1.688452,0.152023,1.040639,0.467767,0.467767,0.493910,0.421824
5,Random Forest,2.427854,1.558157,0.277848,0.876209,0.560111,0.560111,0.502214,0.505851
6,Gradient Boost,2.314163,1.521237,0.311664,0.850931,0.567973,0.567973,0.514871,0.516032
7,XGBoost,2.445997,1.563968,0.272451,0.878387,0.563921,0.563921,0.503958,0.502115
8,AdaBoost,2.586538,1.608272,0.230648,0.920356,0.546807,0.546807,0.493645,0.495031


## Hyperparameter Tuning with GridSearch

In [ ]:
# Define param_grid (TO-DO)
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_clf = RandomForestClassifier()

## Final Evaluation with Testing Set

In [ ]:
# Transform X_test for final evaluation
#X_test_prep = preprocessor.transform(X_test)

In [ ]:
# Predict target with "best model"
#y_pred_rf_clf = best_model_rf_clf.predict(X_test_prep)

In [ ]:
# Final evaluation of "best model"
#print("Accuracy: {:.2f}".format(accuracy_score(y_test, y_pred_rf_clf)))
#print("Recall: {:.2f}".format(recall_score(y_test, y_pred_rf_clf)))
#print("Precision: {:.2f}".format(precision_score(y_test, y_pred_rf_clf)))
#print("F1 Score: {:.2f}".format(f1_score(y_test, y_pred_rf_clf)))
#print("Confusion Matrix: \n" + str(confusion_matrix(y_test, y_pred_rf_clf)))

# Predictive Modeling: Regression ("price")

## Determine Feature Importance

**Apply linear regression**

In [505]:
# Print current setting for TARGET
target_upper = target.upper()
print("You are currently using " + f"\033[1m{target_upper}\033[0m" + " as the target")

You are currently using PRICE_LOG as the target


In [506]:
# Transform X_train_prep and y_train to required format
X_train_prep_ols = X_train_prep.toarray()            # OneHotEncoder outputs csr_matrix, which gives an error when trying to add a constant. Hence, transformed into numpy array
X_train_prep_ols = sm.add_constant(X_train_prep_ols)
y_train_ols = np.asarray(y_train)

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [507]:
# Initialize and fit model
reg_ols = sm.OLS(y_train_ols, X_train_prep_ols).fit()

In [508]:
# Print model summary
reg_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.545
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     86.43
Date:                Wed, 22 Jul 2020   Prob (F-statistic):               0.00
Time:                        18:56:52   Log-Likelihood:                -8469.6
No. Observations:               16914   AIC:                         1.740e+04
Df Residuals:                   16682   BIC:                         1.920e+04
Df Model:                         231                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.9341      0.050     78.766      0.000       3.836       4.032
x1             0.1442      0.005     27.533      0.000       0.134       0.154
x2             0.0272      0.004      7.538      0.000       0.020       0.034
x3             0.0122      0.003      3.847      0.000       0.006       0.018
x4             0.0031      0.003      0.884      0.377      -0.004       0.010
x5             0.0035      0.004      0.905      0.365      -0.004       0.011
x6             0.0379      0.003     11.034      0.000       0.031       0.045
x7             0.0104      0.003      3.196      0.001       0.004       0.017
x8            -0.0009      0.003     -0.282      0.778      -0.007       0.006
x9             0.0169      0.004      4.492      0.000       0.010       0.024
x10           -0.0097      0.003     -3.014      0.003      -0.016      -0.003
x11            0.0249      0.003      7.373      0.000       0.018       0.031
x12           -0.0216      0.003     -6.524      0.000      -0.028      -0.015
x13            0.0394      0.004     11.239      0.000       0.033       0.046
x14            0.0009      0.003      0.279      0.781      -0.005       0.007
x15            0.0988      0.004     25.350      0.000       0.091       0.106
x16            0.0321      0.003      9.490      0.000       0.025       0.039
x17            0.0771      0.009      8.195      0.000       0.059       0.096
x18           -0.0229      0.009     -2.530      0.011      -0.041      -0.005
x19           -0.0315      0.004     -7.536      0.000      -0.040      -0.023
x20            0.0036      0.006      0.656      0.512      -0.007       0.015
x21            0.0042      0.006      0.727      0.467      -0.007       0.016
x22        -3.958e-05      0.004     -0.011      0.991      -0.007       0.007
x23           -0.0459      0.004    -12.724      0.000      -0.053      -0.039
x24            0.0229      0.004      6.264      0.000       0.016       0.030
x25            0.0008      0.004      0.211      0.833      -0.006       0.008
x26           -0.0016      0.003     -0.484      0.629      -0.008       0.005
x27           -0.0078      0.004     -2.036      0.042      -0.015      -0.000
x28           -0.0090      0.009     -0.988      0.323      -0.027       0.009
x29           -0.0044      0.009     -0.493      0.622      -0.022       0.013
x30            0.0076      0.003      2.411      0.016       0.001       0.014
x31           -0.0524      0.003    -15.523      0.000      -0.059      -0.046
x32           -0.0015      0.004     -0.341      0.733      -0.010       0.007
x33            0.0143      0.004      3.595      0.000       0.007       0.022
x34           -0.0028      0.004     -0.779      0.436      -0.010       0.004
x35            0.0484      0.020      2.419      0.016       0.009       0.088
x3

In [509]:
# Predict target (?)

**Apply linear regression w/o preprocessing/differently (to roughly see feature importance)**

In [510]:
# Get num_features as string as basis for model
num_feat_list = str(num_features)
num_feat_list = num_feat_list.strip("[").strip("]").replace("'","").replace(", ", " + ")

In [511]:
# Define target and features for model
model = f'{target} ~ {num_feat_list}'

In [512]:
# Initialize and fit model
reg_ols_wo = smf.ols(formula=model, data=data).fit()

In [513]:
# Print model summary
reg_ols_wo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              price_log   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.378
Method:                 Least Squares   F-statistic:                     512.8
Date:                Wed, 22 Jul 2020   Prob (F-statistic):               0.00
Time:                        18:57:01   Log-Likelihood:                -15450.
No. Observations:               23623   AIC:                         3.096e+04
Df Residuals:                   23594   BIC:                         3.119e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         -2.4444      5.219     -0.468      0.640     -12.674       7.785
accommodates_per_bed               0.0616      0.005     12.707      0.000       0.052       0.071
accommodates_per_room              0.1134      0.003     37.419      0.000       0.107       0.119
am_balcony                         0.0962      0.009     11.240      0.000       0.079       0.113
am_breakfast                       0.0711      0.014      5.262      0.000       0.045       0.098
am_child_friendly                  0.0227      0.007      3.132      0.002       0.008       0.037
am_elevator                        0.1620      0.008     21.469      0.000       0.147       0.177
am_essentials                      0.0607      0.012      5.247      0.000       0.038       0.083
am_nature_and_views               -0.0461      0.020     -2.363      0.018      -0.084      -0.008
am_pets_allowed                   -0.0247      0.008     -2.947      0.003      -0.041      -0.008
am_private_entrance                0.1741      0.009     20.290      0.000       0.157       0.191
am_smoking_allowed                -0.1370      0.008    -16.761      0.000      -0.153      -0.121
am_tv                              0.1674      0.007     25.584      0.000       0.155       0.180
am_white_goods                    -0.0471      0.009     -5.502      0.000      -0.064      -0.030
bathrooms_log                      0.0204      0.015      1.333      0.183      -0.010       0.050
bedrooms                           0.3997      0.006     65.361      0.000       0.388       0.412
calculated_host_listings_count    -0.0024      0.001     -3.592      0.000      -0.004      -0.001
host_is_superhost                  0.0682      0.009      7.215      0.000       0.050       0.087
instant_bookable                   0.0134      0.007      2.025      0.043       0.000       0.026
latitude                           0.1134      0.097      1.168      0.243      -0.077       0.304
longitude                         -0.0579      0.052     -1.115      0.265      -0.160       0.044
maximum_nights                  3.933e-05   5.59e-06      7.031      0.000    2.84e-05    5.03e-05
minimum_nights_log                -0.0267      0.002    -11.076      0.000      -0.031      -0.022
occupancy_class                   -0.0099      0.002     -4.946      0.000      -0.014      -0.006
price_extra_fees_sqrt              0.0073      0.000     19.096      0.000       0.007       0.008
price_extra_people                -0.0048      0.000    -13.323      0.000      -0.006      -0.004
review_scores_rating_sqrt          0.0186      0.002      8.434      0.000       0.014       0.023
text_len_sqrt                      0.0785      0.020      3.869      0.000       0.039       0.118
wk_mth_discoun

**Display Feature Importance (R_Squared) for all features in "data"**

In [514]:
# Select explanatory variables
explanatory_vars = list(X.columns)
explanatory_vars = [e for e in explanatory_vars] 

In [515]:
# Output Top 10 sorted R_Squared among possible features
feat_imp_ols = pd.DataFrame([["baseline", 0.0]])
print('R squared for each possible feature:' )
for explanatory_var in explanatory_vars:
    model = '{target} ~ {feature}'.format(target=target, feature=explanatory_var)
    rs = smf.ols(formula=model, data=data).fit().rsquared
    new_row = pd.DataFrame([[model.split(" ~ ")[-1], rs]])
    feat_imp_ols = pd.concat([new_row, feat_imp_ols], ignore_index=True)
feat_imp_ols.columns = ["FEATURE", "R_SQUARED"]
feat_imp_ols.sort_values(by=["R_SQUARED"], axis=0, ascending=False, inplace=True)
feat_imp_ols.head(20)

R squared for each possible feature:


,FEATURE,R_SQUARED
3,room_type,0.311276
18,bedrooms,0.171072
0,zipcode,0.117642
9,neighbourhood,0.098260
21,am_tv,0.092606
7,price_extra_fees_sqrt,0.063289
23,am_private_entrance,0.047946
31,accommodates_per_room,0.045129
16,cancellation_policy,0.043728
28,am_child_friendly,0.040059


## Apply Regression Models

In [516]:
# Print current setting for TARGET
target_upper = target.upper()
print("You are currently using " + f"\033[1m{target_upper}\033[0m" + " as the target")

You are currently using PRICE_LOG as the target


In [517]:
# Select models for comparison
models={'Baseline': DummyRegressor(strategy='mean'),
        'LinReg': LinearRegression(),
        'Passive Aggressive' : PassiveAggressiveRegressor(),
        'RANSAC' : RANSACRegressor(),
        'ElasticNet' : ElasticNet(),
        'Stochastic Gradient Descent': SGDRegressor(max_iter=1000, tol=1e-3),
        'Decision Tree': DecisionTreeRegressor(criterion="mse", max_depth=3,random_state=random_state),
        'Random Forest': RandomForestRegressor(random_state=random_state, max_features='sqrt', n_jobs=-1),
        'Gradient Boost': GradientBoostingRegressor(random_state=random_state),
        'XGBoost': XGBRegressor(),
        'AdaBoost': AdaBoostRegressor(random_state=random_state)
       }

In [518]:
# Calculate and display results
results = pd.DataFrame(columns=['Model', 'MSE', 'RMSE', 'R2'])
i = 0
for m in models.items():
    # Building a full pipeline with our preprocessor and a Classifier
    pipe = Pipeline([('preprocessor', preprocessor), (m[0], m[1])])
    # Making predictions on the training set using cross validation as well as calculating the probabilities
    y_train_pred = cross_val_predict(pipe,
                                     X_train,
                                     y_train.values.ravel(),
                                     cv=5,
                                     verbose=4,
                                     n_jobs=-1)
    # Calculating metrices
    temp = pd.DataFrame(
        {
            'Model': m[0],
            'MSE': mean_squared_error(y_train, y_train_pred),
            'RMSE': mean_squared_error(
                y_train, y_train_pred, squared=False),
            'MAE': mean_absolute_error(y_train, y_train_pred),
            'R2': r2_score(y_train, y_train_pred)
        },
        index=[i])
    i += 1
    results = pd.concat([results, temp])
results

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    2.4s remaining:    3.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_

,Model,MSE,RMSE,R2,MAE
0,Baseline,0.345622,0.587896,-0.000030,0.467291
1,LinReg,0.185058,0.430183,0.464551,0.332322
2,Passive Aggressive,0.351310,0.592714,-0.016488,0.455845
3,RANSAC,0.350483,0.592016,-0.014096,0.443267
4,ElasticNet,0.345622,0.587896,-0.000030,0.467291
5,Stochastic Gradient Descent,0.187320,0.432805,0.458005,0.334492
6,Decision Tree,0.212829,0.461334,0.384195,0.357838
7,Random Forest,0.154618,0.393215,0.552624,0.299658
8,Gradient Boost,0.154271,0.392774,0.553628,0.300539
9,XGBoost,0.154499,0.393064,0.552969,0.300764


## Hyperparameter Pre-Tuning with RandomizedSearchCV

In [563]:
# Create pipeline to use in GridSearchCV
pipeline_rf_reg = Pipeline([
    ('preprocessor', preprocessor),
    ('rf_reg', RandomForestRegressor(n_estimators=110,
                              random_state=random_state,
                              max_depth=5,
                              max_features=20,
                              n_jobs=-1))
])

In [564]:
# Display possible hyperparameters for RandomForestRegressor
test_rf_reg = RandomForestRegressor()
test_rf_reg.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

**Default values for RandomForestRegressor** (as base for hyperparameter search):

n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None

In [565]:
# Define hyperparameter distribution
param_distribs_rf_reg = {
        'rf_reg__n_estimators': randint(low=10, high=200),
        'rf_reg__max_features': randint(low=1, high=50),
    }

In [566]:
# Create and fit RandomizedSearchCV, save "best_model"
rnd_rf_reg = RandomizedSearchCV(pipeline_rf_reg, param_distribs_rf_reg, cv=5, scoring=scoring, n_iter=20, 
                           return_train_score=True, verbose=4, n_jobs=-1, random_state=random_state)

best_model_rnd_rf_reg = rnd_rf_reg.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.2min finished


In [567]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(rnd_rf_reg.best_score_))
print("Best parameters:\n{}".format(rnd_rf_reg.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
-0.19
Best parameters:
{'rf_reg__max_features': 21, 'rf_reg__n_estimators': 170}


**Determine Feature Importance**

In [568]:
pipeline_rf_reg_fi = pipeline_rf_reg.fit(X_train, y_train)

In [569]:
onehot_columns = list(
    pipeline_rf_reg_fi.named_steps['preprocessor'].named_transformers_['cat'].
    named_steps['1hot'].get_feature_names(input_features=cat_features))
features_prep_list = list(num_features)
features_prep_list.extend(onehot_columns)

In [571]:
eli5.explain_weights(pipeline_rf_reg_fi.named_steps['rf_reg'], top=50, feature_names=features_prep_list)

Weight,Feature
0.4685 ± 0.3670,room_type_Private room
0.2279 ± 0.2896,bedrooms
0.0589 ± 0.1531,am_tv
0.0566 ± 0.1074,accommodates_per_room
0.0478 ± 0.0562,bathrooms_log
0.0303 ± 0.1061,price_extra_fees_sqrt
0.0183 ± 0.0288,room_type_Shared room
0.0168 ± 0.0261,calculated_host_listings_count
0.0114 ± 0.0201,property_type_Boutique hotel
0.0099 ± 0.0196,room_type_Hotel room


## Hyperparameter Tuning with GridSearchCV

In [521]:
# Define hyperparameter grid
param_grid_rf_reg = [
    {'rf_reg__n_estimators': [30, 50, 70], 'rf_reg__max_features': [8, 10, 15, 25]},
    {'rf_reg__bootstrap': [False], 'rf_reg__n_estimators': [30, 50, 70], 'rf_reg__max_features': [8, 10, 15, 25]},
]

In [522]:
# Create and fit GridSearchCV, save "best_model"
grid_rf_reg = GridSearchCV(pipeline_rf_reg, param_grid_rf_reg, cv=5, scoring=scoring, 
                           return_train_score=True, verbose=4, n_jobs=-1, random_state=random_state)

grid_rf_reg.fit(X_train, y_train)
best_model_rf_reg = grid_rf_reg.best_estimator_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   32.0s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   50.7s finished


In [523]:
# Display best_score_, best_params_ and best_estimator_
print('Best score:\n{:.2f}'.format(grid_rf_reg.best_score_))
print("Best parameters:\n{}".format(grid_rf_reg.best_params_))
#print("Best estimator:\n{}".format(grid_rf_reg.best_estimator_))

Best score:
-0.19
Best parameters:
{'rf_reg__max_features': 25, 'rf_reg__n_estimators': 70}


**Feature Importance**

In [524]:
# Get and print feature_importances
#feature_importances = grid_rf_reg.best_estimator_.feature_importances_
#feature_importances
#cat_encoder = preprocessor.named_transformers_["cat"]
#cat_one_hot_features = list(cat_encoder.categories_[0])
#attributes = num_features + cat_one_hot_features
#sorted(zip(feature_importances, attributes), reverse=True)

**Detailed evaluation with training set**

In [529]:
# Predict target with "best model"
y_train_pred_rf_reg = best_model_rf_reg.predict(X_train_prep)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [ ]:
# Final evaluation of "best model"
print("MSE: {:.2f}".format(mean_squared_error(y_test, y_train_pred_rf_reg))),
print("RMSE: {:.2f}".format(mean_squared_error(y_test, y_train_pred_rf_reg, squared=False))),
print("MAE: {:.2f}".format(mean_absolute_error(y_test, y_train_pred_rf_reg))),
print("R2: {:.2f}".format(r2_score(y_test, y_train_pred_rf_reg))),

In [ ]:
# Display confidence interval (scipy stats)
confidence = 0.95
squared_errors = (y_train_pred_rf_reg - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(), 
                          scale=stats.sem(squared_errors)))

## Final evaluation with testing set

In [ ]:
# Transform X_test for final evaluation
#X_test_prep = preprocessor.transform(X_test)

In [ ]:
# Predict target with "best model"
#y_pred_rf_reg = best_model_rf_reg.predict(X_test_prep)

In [ ]:
# Final evaluation of "best model"
#print("MSE: {:.2f}".format(mean_squared_error(y_test, y_pred_rf_reg))),
#print("RMSE: {:.2f}".format(mean_squared_error(y_test, y_pred_rf_reg, squared=False))),
#print("MAE: {:.2f}".format(mean_absolute_error(y_test, y_pred_rf_reg))),
#print("R2: {:.2f}".format(r2_score(y_test, y_pred_rf_reg))),

In [ ]:
# Display confidence interval (scipy stats)
#confidence = 0.95
#squared_errors = (y_pred_rf_reg - y_test) ** 2
#np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
#                         loc=squared_errors.mean(), 
#                         scale=stats.sem(squared_errors)))